In [27]:
import numpy as np
import pandas as pd

In [159]:
df = pd.read_csv('data/single_step_df_ints_2022-04-14_categorized.csv', header=0)

In [160]:
df

,input,target,start_coords.x,start_coords.y,start_coords.z,beverage.already_seen,egg_cup.already_seen,fork_small.already_seen,cookware.already_seen,plate_small.already_seen,...,cookware.food_k,cookware.strong_k,cookware.mid_k,coordinates_egg_cup.x,coordinates_egg_cup.y,coordinates_egg_cup.z,egg_cup.containment,egg_cup.food_k,egg_cup.strong_k,egg_cup.mid_k
0,<start>,plate,-0.451354,-0.413918,0.156247,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,plate,plate_small,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,plate_small,cup,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cup,glass,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,glass,knife,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,<start>,food,1.000000,0.000000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2097,food,cutting_board,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2098,cutting_board,knife,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2099,knife,container,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [161]:
with open('../opportunistic_planning/documentation/legends/item_legend_neural_nets.txt') as legend:
    legend_dict = legend.read()
    item_legend = eval(legend_dict)

In [163]:
def replace_category(df, row, old_category, new_category):
    new_category_containment = str(new_category + '.containment')
    new_category_foodk = str(new_category + '.food_k')
    new_category_midk = str(new_category + '.mid_k')
    new_category_strongk = str(new_category + '.strong_k')
    new_category_alreadyseen = str(new_category + '.already_seen')
    new_category_x = str('coordinates_' + new_category + '.x')
    new_category_y = str('coordinates_' + new_category + '.y')
    new_category_z = str('coordinates_' + new_category + '.z')
    
    old_category_containment = str(old_category + '.containment')
    old_category_foodk = str(old_category + '.food_k')
    old_category_midk = str(old_category + '.mid_k')
    old_category_strongk = str(old_category + '.strong_k')
    old_category_alreadyseen = str(old_category + '.already_seen')
    old_category_x = str('coordinates_' + old_category + '.x')
    old_category_y = str('coordinates_' + old_category + '.y')
    old_category_z = str('coordinates_' + old_category + '.z')
    
    if not new_category_alreadyseen in df.columns:
        df.rename(columns={old_category_containment: new_category_containment,
                           old_category_foodk: new_category_foodk,
                           old_category_midk: new_category_midk,
                           old_category_strongk: new_category_strongk,
                           old_category_alreadyseen: new_category_alreadyseen,
                           old_category_x: new_category_x,
                           old_category_y: new_category_y,
                           old_category_z: new_category_z},
                           inplace=True)
    else:
        pass
    
    return df

In [117]:
def remove_empty_columns(df):
    for col in df.columns:
        if df[col].isnull().all():
            df.drop(col, axis=1, inplace=True)
            
    return df

In [131]:
def remove_old_columns(df, item_legend):
    df_new = df.copy()
    cols_to_drop = []
    
    for col in df.columns:
        for item in item_legend:
            if item in col:
                cols_to_drop.append(col)
                
    df_new.drop(cols_to_drop, axis=1, inplace=True)
    return df_new

In [107]:
def encode_categories(df, item_legend):
    df_new = df.copy()
    
    for row in range(0, len(df)):
        # input item: encode item
        if str(df.at[row, 'input']) != '<start>':
            item = df.at[row, 'input']
            df_new.at[row, 'input'] = item_legend[item]
        
        # target item: encode item + update spatial/parameter info to new category
        if str(df.at[row, 'target']) != '<start>':
            item = df.at[row, 'target']
            replace_category(df_new, row, item, item_legend[item])
            df_new.at[row, 'target'] = item_legend[item]
        
    
    return df_new

In [164]:
df_new = encode_categories(df, item_legend)

In [165]:
df_new

,input,target,start_coords.x,start_coords.y,start_coords.z,h.already_seen,y.already_seen,e.already_seen,v.already_seen,o.already_seen,...,v.food_k,v.strong_k,v.mid_k,coordinates_y.x,coordinates_y.y,coordinates_y.z,y.containment,y.food_k,y.strong_k,y.mid_k
0,<start>,p,-0.451354,-0.413918,0.156247,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p,o,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,o,c,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c,g,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,g,k,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,<start>,d,1.000000,0.000000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2097,d,i,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2098,i,k,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2099,k,x,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
df_new.to_csv('data/single_step_df_ints_2022-04-14_encoded.csv', header=True, index=False)